# python assesment
It's not clear may i use pandas or something else, so i will make two variants

## Extract and transform
first, we need to extract all non empty contracts fields and transform data before we can calculate our features

In [78]:
import csv
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [23]:
# Here we make a flat array with dicts from contracts field
result = {}
cleared = []

with open("data.csv", "r") as f:
    reader = csv.reader(f, delimiter=",")
    for i, line in enumerate(reader):
        if line[2] and i != 0:
            for item in json.loads(line[2]):
                cleared.append(item)

In [24]:
# create Pandas DataFrame and remove duplicates
df = pd.json_normalize(cleared).drop_duplicates()
df.head()

,contract_id,bank,summa,loan_summa,claim_date,claim_id,contract_date
0,522530,003,500000000,0,13.02.2020,609965,17.02.2020
1,,014,,,28.08.2020,F00013731,
2,,014,,,08.10.2020,F00021301,
3,,014,,,25.11.2020,F00037907,
4,,053,,,09.12.2020,34852,


## cast types

In [59]:
df['claim_date'] = pd.to_datetime(df['claim_date'],dayfirst=True)
df['contract_date'] = pd.to_datetime(df['contract_date'],dayfirst=True)
df['summa'] = pd.to_numeric(df['summa'])
df['loan_summa'] = pd.to_numeric(df['loan_summa'])
df.head()

,contract_id,bank,summa,loan_summa,claim_date,claim_id,contract_date
0,522530,003,500000000.0,0.0,2020-02-13,609965,2020-02-17
1,,014,NaN,NaN,2020-08-28,F00013731,NaT
2,,014,NaN,NaN,2020-10-08,F00021301,NaT
3,,014,NaN,NaN,2020-11-25,F00037907,NaT
4,,053,NaN,NaN,2020-12-09,34852,NaT


## Feature 1 (tot_claim_cnt_l180d)

In [60]:
filter_1 = df[(df['claim_date'] >= df['claim_date'].max() - timedelta(days=180)) & (df['claim_date'] != pd.NaT)][['claim_id']]
tot_claim_cnt_l180d = filter_1.count().item()

result['tot_claim_cnt_l180d'] = tot_claim_cnt_l180d if tot_claim_cnt_l180d != 0 else -3
result

{'tot_claim_cnt_l180d': 1029}

## Feature 2 (disb_bank_loan_wo_tbc)

In [66]:
filter_2 = df[(~df['bank'].isin(['LIZ', 'LOM', 'MKO', 'SUG'])) & (df['bank'] != '') & (df['contract_id'] != '')]

if filter_2['claim_id'].count().item() == 0:
    result['disb_bank_loan_wo_tbc'] = -3
elif filter_2['loan_summa'].sum().item() == 0:
    result['disb_bank_loan_wo_tbc'] = -1
else:
    result['disb_bank_loan_wo_tbc'] = filter_2['loan_summa'].sum().item()

result

{'tot_claim_cnt_l180d': 1029, 'disb_bank_loan_wo_tbc': 258665195778.0}

## Feature 3 (day_sinlastloan)

In [85]:
filter_3 = df[~df['summa'].isna()]

last_date = filter_3['contract_date'].max()
if filter_3['claim_id'].count().item() == 0:
    result['day_sinlastloan'] = -3
elif filter_3['loan_summa'].sum().item() == 0:
    result['day_sinlastloan'] = -1
else:
    result['day_sinlastloan'] = int((datetime.now() - last_date) / timedelta(days=1))

result


{'tot_claim_cnt_l180d': 1029,
 'disb_bank_loan_wo_tbc': 258665195778.0,
 'day_sinlastloan': 202}

## Write result to csv file

In [86]:
result = pd.json_normalize(result)
result.to_csv('contract_features.csv', encoding='utf-8', index=False)